In [1]:
import numpy as np
from tvm.relay import testing
import tvm
from tvm import te
from tvm.contrib import graph_executor
import tvm.testing
import tensorflow as tf
import numpy as np
import tvm.relay as relay
from tvm import relay
from tvm.relay.build_module import bind_params_by_name
from tvm.relay.dataflow_pattern import *

import json
import numpy as np
import pygraphviz as pgv
PATH_MODEL = './model/'
def show_graph(json_data, file_name=None):
    if type(json_data) == str:
        json_data = json.loads(json_data)
    A = pgv.AGraph(directed=True)
    for node_idx, node in enumerate(json_data['nodes']):
        for src in node['inputs']:
            A.add_edge(json_data['nodes'][src[0]]['name'] + '[{}]'.format(src[0]) + '{}'.format(json_data['attrs']['dltype'][1][src[0]]), node['name'] + '[{}]'.format(node_idx) + '{}'.format(json_data['attrs']['dltype'][1][node_idx]))
            #A.add_edge(json_data['nodes'][src[0]]['name'] + '[{}]'.format(src[0]) + '{}'.format(shape_size(json_data['attrs']['shape'][1][src[0]])) + '{}'.format(json_data['attrs']['dltype'][1][src[0]]), node['name'] + '[{}]'.format(node_idx) + '{}'.format(shape_size(json_data['attrs']['shape'][1][node_idx])) + '{}'.format(json_data['attrs']['dltype'][1][src[0]]))
    if file_name:
        A.draw(file_name + '.png', format='png', prog='dot')

2022-05-07 22:06:28.472760: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
model_keras = tf.keras.models.load_model('../../tvm-slicer/src/model/{}_{}.h5'.format("unet", 512))

input_data = np.random.normal(0,1,(1,512,512,3)).astype(np.float32)
input_data = input_data.transpose([0, 3, 1, 2])
shape_dict = {"input_1": input_data.shape}
mod, params = relay.frontend.from_keras(model_keras, shape_dict)

2022-05-07 22:06:45.365440: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-05-07 22:06:45.365505: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-05-07 22:06:45.393177: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-07 22:06:45.393459: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.665GHz coreCount: 68 deviceMemorySize: 10.75GiB deviceMemoryBandwidth: 573.69GiB/s
2022-05-07 22:06:45.393473: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-05-07 22:06:45.395170: I tensorflow/stream_executor/platfor

In [3]:
print(mod)

def @main(%input_1: Tensor[(1, 3, 512, 512), float32], %v_param_1: Tensor[(16, 3, 3, 3), float32], %v_param_2: Tensor[(16), float32], %v_param_3: Tensor[(16), float32], %v_param_4: Tensor[(16), float32], %v_param_5: Tensor[(16), float32], %v_param_6: Tensor[(16), float32], %v_param_7: Tensor[(16, 16, 3, 3), float32], %v_param_8: Tensor[(16), float32], %v_param_9: Tensor[(16), float32], %v_param_10: Tensor[(16), float32], %v_param_11: Tensor[(16), float32], %v_param_12: Tensor[(16), float32], %v_param_13: Tensor[(32, 16, 3, 3), float32], %v_param_14: Tensor[(32), float32], %v_param_15: Tensor[(32), float32], %v_param_16: Tensor[(32), float32], %v_param_17: Tensor[(32), float32], %v_param_18: Tensor[(32), float32], %v_param_19: Tensor[(32, 32, 3, 3), float32], %v_param_20: Tensor[(32), float32], %v_param_21: Tensor[(32), float32], %v_param_22: Tensor[(32), float32], %v_param_23: Tensor[(32), float32], %v_param_24: Tensor[(32), float32], %v_param_25: Tensor[(64, 32, 3, 3), float32], %v_pa

In [130]:
is_conv2d = is_op("nn.conv2d")(wildcard(), wildcard())
is_max_pool2d = is_op("nn.max_pool2d")(wildcard(), wildcard())
is_batch_norm = is_op("nn.batch_norm")(wildcard(), wildcard(), wildcard(), wildcard(), wildcard())
is_leaky_relu = is_op("nn.leaky_relu")(wildcard())
is_sigmoid = is_op("sigmoid")(wildcard())
# var2 = wildcard()
# tuple_node = is_tuple([wildcard(), var2])
is_concat = is_op('concatenate')(wildcard())
is_bias_add = is_op("nn.bias_add")(wildcard(), wildcard())

In [131]:
var2 = wildcard()
tuple_node = is_tuple([wildcard(), var2])
concat_node = is_op('concatenate')(tuple_node)

In [132]:
diamond = dominates(is_conv2d, wildcard(), is_batch_norm)

In [137]:
inp = relay.var("input")
weight = relay.var("weight")
bias = relay.var("bias")
conv2d = relay.op.nn.conv2d(inp, weight)
relu = relay.op.nn.relu(conv2d)
leaky_relu = relay.op.nn.leaky_relu(relu, alpha=0.2)
badd = relay.op.nn.bias_add(leaky_relu, bias)
relu = relay.op.nn.relu(badd)
leaky_relu = relay.op.nn.leaky_relu(relu, alpha=0.2)
badd = relay.op.nn.bias_add(leaky_relu, bias)
relu = relay.op.nn.relu(badd)
leaky_relu = relay.op.nn.leaky_relu(relu, alpha=0.2)
badd = relay.op.nn.bias_add(leaky_relu, bias)
relu = relay.op.nn.relu(badd)
relu = relay.op.nn.relu(relu)
out = relu

In [52]:
out

CallNode(Op(nn.relu), [CallNode(Op(nn.relu), [CallNode(Op(nn.leaky_relu), [CallNode(Op(nn.relu), [CallNode(Op(nn.leaky_relu), [CallNode(Op(nn.relu), [CallNode(Op(nn.leaky_relu), [CallNode(Op(nn.relu), [CallNode(Op(nn.conv2d), [Var(input), Var(weight)], relay.attrs.Conv2DAttrs(0x1c148a78), [])], (nullptr), [])], relay.attrs.LeakyReluAttrs(0x4a3a2e8), [])], (nullptr), [])], relay.attrs.LeakyReluAttrs(0x1b7c67d8), [])], (nullptr), [])], relay.attrs.LeakyReluAttrs(0x643d508), [])], (nullptr), [])], (nullptr), [])

In [138]:
print(out)

free_var %input;
free_var %weight;
%0 = nn.conv2d(%input, %weight, padding=[0, 0, 0, 0]);
%1 = nn.relu(%0);
%2 = nn.leaky_relu(%1, alpha=0.2f);
free_var %bias;
%3 = nn.bias_add(%2, %bias);
%4 = nn.relu(%3);
%5 = nn.leaky_relu(%4, alpha=0.2f);
%6 = nn.bias_add(%5, %bias);
%7 = nn.relu(%6);
%8 = nn.leaky_relu(%7, alpha=0.2f);
%9 = nn.bias_add(%8, %bias);
%10 = nn.relu(%9);
nn.relu(%10)


In [140]:
is_conv2d = is_op("nn.conv2d")(wildcard(), wildcard())
# is_unary_elemwise = (wildcard().has_attr({"TOpPattern": K_ELEMWISE}))(wildcard())
# reduction = is_op("add")(wildcard(), wildcard())
is_leaky_relu = is_op("nn.leaky_relu")(wildcard())
is_relu = is_op("nn.relu")(wildcard())
is_bias_add = is_op("nn.bias_add")(wildcard(), wildcard())
# diamond = dominates(is_conv2d, wildcard(), reduction)
diamond = dominates(is_conv2d, wildcard(), is_leaky_relu)


In [141]:
diamond.match(out)

0

In [142]:
print(out)
print("===================")
print(diamond.partition(out))

free_var %input;
free_var %weight;
%0 = nn.conv2d(%input, %weight, padding=[0, 0, 0, 0]);
%1 = nn.relu(%0);
%2 = nn.leaky_relu(%1, alpha=0.2f);
free_var %bias;
%3 = nn.bias_add(%2, %bias);
%4 = nn.relu(%3);
%5 = nn.leaky_relu(%4, alpha=0.2f);
%6 = nn.bias_add(%5, %bias);
%7 = nn.relu(%6);
%8 = nn.leaky_relu(%7, alpha=0.2f);
%9 = nn.bias_add(%8, %bias);
%10 = nn.relu(%9);
nn.relu(%10)
free_var %input;
free_var %weight;
%2 = fn (%FunctionVar_2_0, %FunctionVar_2_1, PartitionedFromPattern="nn.conv2d_nn.relu_nn.leaky_relu_") {
  %0 = nn.conv2d(%FunctionVar_2_0, %FunctionVar_2_1, padding=[0, 0, 0, 0]);
  %1 = nn.relu(%0);
  nn.leaky_relu(%1, alpha=0.2f)
};
%3 = %2(%input, %weight);
free_var %bias;
%4 = nn.bias_add(%3, %bias);
%5 = nn.relu(%4);
%6 = nn.leaky_relu(%5, alpha=0.2f);
%7 = nn.bias_add(%6, %bias);
%8 = nn.relu(%7);
%9 = nn.leaky_relu(%8, alpha=0.2f);
%10 = nn.bias_add(%9, %bias);
%11 = nn.relu(%10);
nn.relu(%11)


In [82]:
inp = relay.var('input')
weight = relay.var('weight')
conv2d = relay.op.nn.conv2d(inp, weight)
relu = relay.op.nn.relu(conv2d)
leaky_relu = relay.op.nn.leaky_relu(conv2d, alpha=0)
out = relu - leaky_relu
conv2d = relay.op.nn.conv2d(out, weight)
relu = relay.op.nn.relu(conv2d)
leaky_relu = relay.op.nn.leaky_relu(conv2d, alpha=0)
out = relu + leaky_relu


In [83]:
print(out)

free_var %input;
free_var %weight;
%0 = nn.conv2d(%input, %weight, padding=[0, 0, 0, 0]);
%1 = nn.relu(%0);
%2 = nn.leaky_relu(%0, alpha=0f);
%3 = subtract(%1, %2);
%4 = nn.conv2d(%3, %weight, padding=[0, 0, 0, 0]);
%5 = nn.relu(%4);
%6 = nn.leaky_relu(%4, alpha=0f);
add(%5, %6)


In [96]:
is_conv2d = is_op('nn.conv2d')(is_var(), is_var())
reduction = is_op('add')(wildcard(), wildcard())
# diamond = dominates(is_conv2d,  is_op('subtract')(wildcard(), wildcard()), reduction)
diamond = dominates(is_conv2d, wildcard(), reduction)


In [97]:
diamond.match(out)

1

In [98]:
print(diamond.partition(out))

free_var %input;
%7 = fn (%FunctionVar_0_0, PartitionedFromPattern="nn.conv2d_nn.relu_nn.leaky_relu_subtract_nn.conv2d_nn.relu_nn.leaky_relu_add_") {
  free_var %weight;
  %0 = nn.conv2d(%FunctionVar_0_0, %weight, padding=[0, 0, 0, 0]);
  %1 = nn.relu(%0);
  %2 = nn.leaky_relu(%0, alpha=0f);
  %3 = subtract(%1, %2);
  %4 = nn.conv2d(%3, %weight, padding=[0, 0, 0, 0]);
  %5 = nn.relu(%4);
  %6 = nn.leaky_relu(%4, alpha=0f);
  add(%5, %6)
};
%7(%input)
